In [ ]:
# libraries
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from scipy.ndimage import convolve, generate_binary_structure
from numba import njit

In [ ]:
# @njit(nopython = True, nogil = True)
def Lat_Energy(lattice, j1, j2):
    kernal = np.matrix([[0., 0., j2, 0., 0.], 
                        [0., j1, 0., j1, 0.], 
                        [j2, 0., 0., 0., j2],
                        [0., j1, 0., j1, 0.], 
                        [0., 0., j2, 0., 0.]], dtype = "float")
    arr = -lattice*convolve(lattice, kernal, mode = 'wrap', cval = 0, output = "float")
    # print(arr)
    # arr = arr/2
    '''use mode = 'wrap' if considering infinite/periodic lattice'''
    return np.sum(arr/2)

In [ ]:
arr = np.array([[ 0, -1,  0,  1,  0, -1,  0,  1,  0, -1],
               [ 1,  0, -1,  0,  1,  0, -1,  0,  1,  0],
               [ 0,  1,  0, -1,  0,  1,  0, -1,  0,  1,],
               [-1,  0,  1,  0, -1,  0,  1,  0, -1,  0,],
               [ 0, -1,  0,  1,  0, -1,  0,  1,  0, -1],
               [ 1,  0, -1,  0,  1,  0, -1,  0,  1,  0],
               [ 0,  1,  0, -1,  0,  1,  0, -1,  0,  1],
               [-1,  0,  1,  0, -1,  0,  1,  0, -1,  0],
               [ 0, -1,  0,  1,  0, -1,  0,  1,  0, -1],
               [ 1,  0, -1,  0,  1,  0, -1,  0,  1,  0]])

# arr3 = arr[arr != 0].reshape(10, 5)
# plt.imshow(arr)
# print(stagger(arr))

# print(Lat_Energy(arr, 2.3*10**(-3), -21*10**(-3))/50)

In [ ]:
# arr2 = np.matrix([[0., 0., 0, 0., 0.], 
#                         [0., 0, 0., 0, 0.], 
#                         [0, 0., 1, 0., 0],
#                         [0., 0, 0., 0, 0.], 
#                         [0., 0., 0, 0., 0.]])
arr2 = np.array([[1, 0], 
                [0, -1]])
print(Lat_Energy(arr2, 2.3, -21))

In [ ]:
def NiO_shape(lattice):
    for i in range(len(lattice)):
        for j in range(len(lattice)):
            if(i%2 == 0):
                if(j%2 == 0):
                    lattice[i, j] = 0
            if(i%2 == 1):
                if(j%2 == 1):
                    lattice[i, j] = 0      
    return(lattice)  

In [ ]:
@njit(nopython = True, nogil = True)
def stagger(lat_config):
    size = len(lat_config)
    rows = np.zeros(size)

    for i in range(size):
        row_dat = 0

        for j in range(size):

            if(i%2 == 0):
                if(((j+1)%2 == 0) and ((j+1)%4 != 0)):
                    row_dat += lat_config[i, j]
                elif((j+1)%4 == 0):
                    row_dat -= lat_config[i, j]

            else:
                if((j%2 == 0) and (j%4 != 0)):
                    row_dat -= lat_config[i, j]
                elif(j%4 == 0):
                    row_dat += lat_config[i, j]

        rows[i] = row_dat

        res = 0
        for data in rows:
            res += np.abs(data)

    return res, rows

In [ ]:
@njit(nopython = True, nogil = True)

def metropolis(lat_config, n_times, beta, lattice_energy, N, J1, J2):
    spin_arr = lat_config.copy()
    net_spins = np.zeros(n_times-1)
    net_lattice_energy = np.zeros(n_times-1)

    for t in range(0, n_times-1):

        x, y = 0, 0
        while(spin_arr[x, y] == 0):
          x, y = np.random.randint(low = 0, high = N, size = 2)

        spin_i = spin_arr[x, y]     # initial spin
        spin_f = spin_i*(-1)        # flipping initial spin
        
        E_i = 0
        E_f = 0

        d1 = spin_arr[(x-1)%N, (y-1)%N]
        d2 = spin_arr[(x+1)%N, (y+1)%N]
        d3 = spin_arr[(x+1)%N, (y-1)%N]
        d4 = spin_arr[(x-1)%N, (y+1)%N]

        l1 = spin_arr[(x)%N, (y-2)%N]
        l2 = spin_arr[(x)%N, (y+2)%N]
        l3 = spin_arr[(x+2)%N, (y)%N]
        l4 = spin_arr[(x-2)%N, (y)%N]

        E_i = -J1*spin_i*(d1+d2+d3+d4) - J2*spin_i*(l1+l2+l3+l4) 
        E_f = -J1*spin_f*(d1+d2+d3+d4) - J2*spin_f*(l1+l2+l3+l4) 

        dE = E_f - E_i

        if ((dE > 0) and (np.random.uniform(0, 1) < np.exp(-beta*dE))):
            spin_arr[x, y] = spin_f
            lattice_energy += dE

        elif dE <= 0:
            spin_arr[x, y] = spin_f
            lattice_energy += dE
            
        net_spins[t] = stagger(spin_arr)[0]
        net_lattice_energy[t] = lattice_energy

    final_config = spin_arr.copy()
            
    return net_spins, net_lattice_energy, final_config

In [ ]:
def get_spin_energy(lattice, beta, correct_n, J1, J2, N):
    ms = np.zeros(len(beta))         # array for means of spins
    m_stds = np.zeros(len(beta))
    e_means = np.zeros(len(beta))
    e_stds = np.zeros(len(beta))

    for i, b in enumerate(beta):
        energy = Lat_Energy(lattice, J1, J2)
        spins, energies, final = metropolis(lattice, 10**6, b, energy, N, J1, J2)

        # correct_n is the number of spins/particles (from the end of the array) that we'd like to consider to calc. the mean and std. It is like convergence
        ms[i] = np.mean(spins[-correct_n:])/(N**2/2) #.mean()/N**2
        m_stds[i] = np.std(spins[-correct_n:])
        e_means[i] = np.mean(energies[-correct_n:])/(N**2/2)#.mean()
        e_stds[i] = np.std(energies[-correct_n:])#.std()

        print(i)

    return ms, m_stds, e_means, e_stds

In [ ]:
'''Lattice Information'''
Ndim = 20              # NxN lattice
p = 0.50            # Prob of -1 spins
lattice = NiO_shape(np.random.choice([-1, 1], size = (Ndim, Ndim), p = [p, 1-p]))

plt.figure("org_lattice")
plt.imshow(lattice)

In [ ]:
n_steps = 10**6     # Number of times to run metropolis: (randomly) flip spins
j1 = 2.3*10**(-3)
j2 = -21*10**(-3)

energy_lattice = Lat_Energy(lattice, j1, j2)
kb = 8.6173303*10**(-5)
T = 100
B = 1/(T*kb)

spin, enrgy, final = metropolis(lat_config = lattice, n_times = n_steps, beta = B, lattice_energy = energy_lattice, N = Ndim, J1 = j1, J2 = j2)

# plt.figure("final_lattice")
# plt.imshow(final)

In [37]:
plt.figure("lattice")
plt.imshow(final)

In [ ]:
'''Plot of Spins and Energy for particular BJ'''
fig, ax = plt.subplots(1, 2, figsize = (13.5, 5.5)); 
ax[0].plot(spin/(Ndim**2))
ax[0].set_xlabel("n Steps")
ax[0].set_ylabel("Average Staggered Magnetization $\\bar{m}$")
ax[0].grid(True)
ax[1].plot(1000*enrgy/(Ndim**2))
ax[1].set_xlabel("n Steps")
ax[1].set_ylabel("Average Energy $E/J$")
ax[1].grid(True)
fig.suptitle("Average Spin and Energy for $\\beta J = $ {:}".format(B), size = 12); 

<hr/>

In [ ]:
kb = 8.6173303*10**(-5)
beta = 1/(kb*np.linspace(50, 1000, 200))
j1 = 2.3*10**(-3)
j2 = -21*10**(-3)
s_means, s_stds, E_means, E_stds = get_spin_energy(lattice, beta, 10**5, j1, j2, Ndim)

In [ ]:
'''Plot of Magnetization and Cv vs 1/BJ'''
fig2, ax2 = plt.subplots(1, 2, figsize = (13.5, 5.5)); 
# ms_max_ind = np.argmax(s_means)
ax2[0].plot(1/(beta*kb), s_means, ls = '--', marker = ".")
# ax2[0].axvline(1/BJs[ms_max_ind], color = "black", label = "1/BJ = {:.5}".format(1/BJs[ms_max_ind]))
ax2[0].set_xlabel("Temp (K)")
ax2[0].set_ylabel("$\\bar{sm}$")
ax2[0].grid()
# ax2[0].legend()

cv_max_ind = np.argmax(E_stds*(beta**2))
ax2[1].plot(1/(beta*kb), E_stds*(beta**2), ls = "-.", marker = "o")
ax2[1].axvline(1/(kb*beta[cv_max_ind]), color = "black", label = "1/BJ = {:.5}".format(1/(kb*beta[cv_max_ind])))
ax2[1].set_xlabel("Temp (K)")
ax2[1].set_ylabel("$C_V / k^2$")
ax2[1].grid()
ax2[1].legend()
fig2.suptitle("Mean Staggered Magnetization and Specific Heat\n{:}% of spins started negative".format(int(p*100))); 

In [ ]:
np.savetxt(r"C:\Users\jains\Desktop\nio200_new.txt", np.column_stack([s_means, s_stds, E_means, E_stds]))

<hr/>